## 데이타셋 만들기

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
train_datasets = tfds.load('cifar10', split='train')
valid_datasets = tfds.load('cifar10', split='test')

In [3]:
train_datasets

<PrefetchDataset shapes: {id: (), image: (32, 32, 3), label: ()}, types: {id: tf.string, image: tf.uint8, label: tf.int64}>

In [4]:
def preprocessing(data):
    image = tf.cast(data['image'], tf.float32) / 255.0
    label = data['label']
    return image, label

In [5]:
BATCH_SIZE = 128
train_data = train_datasets.map(preprocessing).shuffle(100).batch(BATCH_SIZE)
valid_data = valid_datasets.map(preprocessing).batch(BATCH_SIZE)

In [6]:
for image, label in train_data.take(1):
    print(image.shape)
    print(label.shape)

(128, 32, 32, 3)
(128,)


## Sequential

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [8]:
def sequential_model(h, w, c):
    model = Sequential([
        Conv2D(32, 3, activation='relu', input_shape=(h, w, c)),
        MaxPooling2D(2, 2),
        Conv2D(64, 3, activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(32, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model

In [9]:
seq_model = sequential_model(32, 32, 3)
seq_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                73760     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3

In [10]:
seq_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [11]:
seq_model.fit(train_data,
              validation_data=(valid_data),
              epochs=10,
             )

Epoch 1/10
391/391 [==============================] - 17s 20ms/step - loss: 1.8902 - acc: 0.3098 - val_loss: 1.4007 - val_acc: 0.4886
Epoch 2/10
391/391 [==============================] - 5s 14ms/step - loss: 1.3885 - acc: 0.5005 - val_loss: 1.2714 - val_acc: 0.5480
Epoch 3/10
391/391 [==============================] - 5s 14ms/step - loss: 1.2453 - acc: 0.5611 - val_loss: 1.1650 - val_acc: 0.5914
Epoch 4/10
391/391 [==============================] - 6s 14ms/step - loss: 1.1474 - acc: 0.5999 - val_loss: 1.1004 - val_acc: 0.6126
Epoch 5/10
391/391 [==============================] - 6s 14ms/step - loss: 1.0722 - acc: 0.6263 - val_loss: 1.0443 - val_acc: 0.6335
Epoch 6/10
391/391 [==============================] - 5s 14ms/step - loss: 1.0134 - acc: 0.6493 - val_loss: 1.0062 - val_acc: 0.6507
Epoch 7/10
391/391 [==============================] - 6s 14ms/step - loss: 0.9672 - acc: 0.6666 - val_loss: 0.9722 - val_acc: 0.6629
Epoch 8/10
391/391 [==============================] - 6s 14ms/step -

## Functional API

In [12]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [13]:
def functional_model(h, w, c):
    input_ = Input(shape=((h, w, c)))
    x = Conv2D(32, 3, activation='relu')(input_)
    x = MaxPooling2D(2, 2)(x)
    x = Conv2D(64, 3, activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    x = Flatten()(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(input_, x)
    return model

In [14]:
func_model = functional_model(32, 32, 3)
func_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                73760 

In [15]:
func_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [16]:
func_model.fit(train_data,
              validation_data=(valid_data),
              epochs=10,
             )

Epoch 1/10
391/391 [==============================] - 6s 15ms/step - loss: 1.8710 - acc: 0.3197 - val_loss: 1.3759 - val_acc: 0.5103
Epoch 2/10
391/391 [==============================] - 6s 14ms/step - loss: 1.3379 - acc: 0.5288 - val_loss: 1.2087 - val_acc: 0.5740
Epoch 3/10
391/391 [==============================] - 5s 14ms/step - loss: 1.1943 - acc: 0.5834 - val_loss: 1.1485 - val_acc: 0.5924
Epoch 4/10
391/391 [==============================] - 6s 14ms/step - loss: 1.1114 - acc: 0.6132 - val_loss: 1.0800 - val_acc: 0.6211
Epoch 5/10
391/391 [==============================] - 6s 14ms/step - loss: 1.0457 - acc: 0.6363 - val_loss: 1.0283 - val_acc: 0.6412
Epoch 6/10
391/391 [==============================] - 6s 14ms/step - loss: 0.9945 - acc: 0.6557 - val_loss: 1.0050 - val_acc: 0.6528
Epoch 7/10
391/391 [==============================] - 6s 14ms/step - loss: 0.9519 - acc: 0.6713 - val_loss: 0.9849 - val_acc: 0.6579
Epoch 8/10
391/391 [==============================] - 6s 14ms/step - 

## train_on_batch

In [17]:
tb_model = functional_model(32, 32, 3)

In [18]:
tb_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [19]:
EPOCHS = 10

for epoch in range(EPOCHS):
    for batch, (image, label) in train_data.enumerate():
        loss = tb_model.train_on_batch(image, label)
        print(f'epoch: {epoch + 1}, batch: {batch + 1}, loss:{loss[0]:.3f}, acc: {loss[1]:.2f}')

epoch: 1, batch: 1, loss:2.304, acc: 0.09
epoch: 1, batch: 2, loss:2.300, acc: 0.12
epoch: 1, batch: 3, loss:2.264, acc: 0.09
epoch: 1, batch: 4, loss:2.305, acc: 0.14
epoch: 1, batch: 5, loss:2.319, acc: 0.09
epoch: 1, batch: 6, loss:2.266, acc: 0.11
epoch: 1, batch: 7, loss:2.330, acc: 0.15
epoch: 1, batch: 8, loss:2.269, acc: 0.14
epoch: 1, batch: 9, loss:2.265, acc: 0.20
epoch: 1, batch: 10, loss:2.284, acc: 0.14
epoch: 1, batch: 11, loss:2.285, acc: 0.11
epoch: 1, batch: 12, loss:2.280, acc: 0.15
epoch: 1, batch: 13, loss:2.279, acc: 0.09
epoch: 1, batch: 14, loss:2.269, acc: 0.13
epoch: 1, batch: 15, loss:2.280, acc: 0.09
epoch: 1, batch: 16, loss:2.267, acc: 0.10
epoch: 1, batch: 17, loss:2.254, acc: 0.19
epoch: 1, batch: 18, loss:2.243, acc: 0.22
epoch: 1, batch: 19, loss:2.240, acc: 0.24
epoch: 1, batch: 20, loss:2.243, acc: 0.23
epoch: 1, batch: 21, loss:2.210, acc: 0.19
epoch: 1, batch: 22, loss:2.241, acc: 0.19
epoch: 1, batch: 23, loss:2.241, acc: 0.16
epoch: 1, batch: 24,

epoch: 1, batch: 194, loss:1.460, acc: 0.52
epoch: 1, batch: 195, loss:1.766, acc: 0.34
epoch: 1, batch: 196, loss:1.715, acc: 0.42
epoch: 1, batch: 197, loss:1.580, acc: 0.43
epoch: 1, batch: 198, loss:1.555, acc: 0.38
epoch: 1, batch: 199, loss:1.590, acc: 0.44
epoch: 1, batch: 200, loss:1.539, acc: 0.50
epoch: 1, batch: 201, loss:1.569, acc: 0.46
epoch: 1, batch: 202, loss:1.686, acc: 0.41
epoch: 1, batch: 203, loss:1.584, acc: 0.39
epoch: 1, batch: 204, loss:1.680, acc: 0.41
epoch: 1, batch: 205, loss:1.581, acc: 0.40
epoch: 1, batch: 206, loss:1.567, acc: 0.44
epoch: 1, batch: 207, loss:1.602, acc: 0.48
epoch: 1, batch: 208, loss:1.555, acc: 0.48
epoch: 1, batch: 209, loss:1.640, acc: 0.39
epoch: 1, batch: 210, loss:1.626, acc: 0.41
epoch: 1, batch: 211, loss:1.469, acc: 0.50
epoch: 1, batch: 212, loss:1.579, acc: 0.42
epoch: 1, batch: 213, loss:1.661, acc: 0.47
epoch: 1, batch: 214, loss:1.662, acc: 0.42
epoch: 1, batch: 215, loss:1.582, acc: 0.40
epoch: 1, batch: 216, loss:1.602

epoch: 1, batch: 383, loss:1.358, acc: 0.54
epoch: 1, batch: 384, loss:1.429, acc: 0.53
epoch: 1, batch: 385, loss:1.375, acc: 0.51
epoch: 1, batch: 386, loss:1.313, acc: 0.51
epoch: 1, batch: 387, loss:1.406, acc: 0.55
epoch: 1, batch: 388, loss:1.445, acc: 0.48
epoch: 1, batch: 389, loss:1.356, acc: 0.60
epoch: 1, batch: 390, loss:1.238, acc: 0.56
epoch: 1, batch: 391, loss:1.337, acc: 0.51
epoch: 2, batch: 1, loss:1.667, acc: 0.41
epoch: 2, batch: 2, loss:1.290, acc: 0.49
epoch: 2, batch: 3, loss:1.390, acc: 0.49
epoch: 2, batch: 4, loss:1.485, acc: 0.46
epoch: 2, batch: 5, loss:1.588, acc: 0.46
epoch: 2, batch: 6, loss:1.510, acc: 0.50
epoch: 2, batch: 7, loss:1.329, acc: 0.48
epoch: 2, batch: 8, loss:1.623, acc: 0.43
epoch: 2, batch: 9, loss:1.358, acc: 0.51
epoch: 2, batch: 10, loss:1.242, acc: 0.57
epoch: 2, batch: 11, loss:1.465, acc: 0.44
epoch: 2, batch: 12, loss:1.519, acc: 0.47
epoch: 2, batch: 13, loss:1.492, acc: 0.43
epoch: 2, batch: 14, loss:1.436, acc: 0.45
epoch: 2, b

epoch: 2, batch: 182, loss:1.305, acc: 0.53
epoch: 2, batch: 183, loss:1.215, acc: 0.55
epoch: 2, batch: 184, loss:1.257, acc: 0.52
epoch: 2, batch: 185, loss:1.345, acc: 0.52
epoch: 2, batch: 186, loss:1.372, acc: 0.51
epoch: 2, batch: 187, loss:1.375, acc: 0.50
epoch: 2, batch: 188, loss:1.214, acc: 0.56
epoch: 2, batch: 189, loss:1.326, acc: 0.54
epoch: 2, batch: 190, loss:1.319, acc: 0.52
epoch: 2, batch: 191, loss:1.332, acc: 0.49
epoch: 2, batch: 192, loss:1.275, acc: 0.55
epoch: 2, batch: 193, loss:1.266, acc: 0.55
epoch: 2, batch: 194, loss:1.144, acc: 0.64
epoch: 2, batch: 195, loss:1.433, acc: 0.48
epoch: 2, batch: 196, loss:1.270, acc: 0.55
epoch: 2, batch: 197, loss:1.296, acc: 0.53
epoch: 2, batch: 198, loss:1.304, acc: 0.47
epoch: 2, batch: 199, loss:1.396, acc: 0.55
epoch: 2, batch: 200, loss:1.360, acc: 0.49
epoch: 2, batch: 201, loss:1.242, acc: 0.58
epoch: 2, batch: 202, loss:1.382, acc: 0.51
epoch: 2, batch: 203, loss:1.365, acc: 0.50
epoch: 2, batch: 204, loss:1.214

epoch: 2, batch: 375, loss:1.111, acc: 0.62
epoch: 2, batch: 376, loss:1.296, acc: 0.52
epoch: 2, batch: 377, loss:1.397, acc: 0.48
epoch: 2, batch: 378, loss:1.246, acc: 0.58
epoch: 2, batch: 379, loss:1.209, acc: 0.58
epoch: 2, batch: 380, loss:1.176, acc: 0.56
epoch: 2, batch: 381, loss:1.329, acc: 0.57
epoch: 2, batch: 382, loss:1.292, acc: 0.47
epoch: 2, batch: 383, loss:1.161, acc: 0.59
epoch: 2, batch: 384, loss:1.113, acc: 0.62
epoch: 2, batch: 385, loss:1.240, acc: 0.63
epoch: 2, batch: 386, loss:1.324, acc: 0.58
epoch: 2, batch: 387, loss:1.146, acc: 0.62
epoch: 2, batch: 388, loss:1.230, acc: 0.55
epoch: 2, batch: 389, loss:1.169, acc: 0.59
epoch: 2, batch: 390, loss:1.181, acc: 0.58
epoch: 2, batch: 391, loss:1.119, acc: 0.70
epoch: 3, batch: 1, loss:1.412, acc: 0.48
epoch: 3, batch: 2, loss:1.168, acc: 0.56
epoch: 3, batch: 3, loss:1.337, acc: 0.53
epoch: 3, batch: 4, loss:1.252, acc: 0.57
epoch: 3, batch: 5, loss:1.323, acc: 0.56
epoch: 3, batch: 6, loss:1.347, acc: 0.57


...

epoch: 8, batch: 341, loss:0.863, acc: 0.70
epoch: 8, batch: 342, loss:0.857, acc: 0.69
epoch: 8, batch: 343, loss:1.048, acc: 0.68
epoch: 8, batch: 344, loss:0.770, acc: 0.74
epoch: 8, batch: 345, loss:0.898, acc: 0.73
epoch: 8, batch: 346, loss:0.972, acc: 0.70
epoch: 8, batch: 347, loss:0.848, acc: 0.77
epoch: 8, batch: 348, loss:0.979, acc: 0.66
epoch: 8, batch: 349, loss:0.986, acc: 0.69
epoch: 8, batch: 350, loss:0.860, acc: 0.64
epoch: 8, batch: 351, loss:0.961, acc: 0.66
epoch: 8, batch: 352, loss:1.042, acc: 0.62
epoch: 8, batch: 353, loss:0.913, acc: 0.66
epoch: 8, batch: 354, loss:0.952, acc: 0.67
epoch: 8, batch: 355, loss:0.751, acc: 0.75
epoch: 8, batch: 356, loss:1.006, acc: 0.64
epoch: 8, batch: 357, loss:0.873, acc: 0.73
epoch: 8, batch: 358, loss:0.842, acc: 0.68
epoch: 8, batch: 359, loss:0.949, acc: 0.66
epoch: 8, batch: 360, loss:1.017, acc: 0.62
epoch: 8, batch: 361, loss:1.013, acc: 0.64
epoch: 8, batch: 362, loss:0.938, acc: 0.65
epoch: 8, batch: 363, loss:1.059

epoch: 9, batch: 144, loss:0.949, acc: 0.67
epoch: 9, batch: 145, loss:0.811, acc: 0.70
epoch: 9, batch: 146, loss:0.883, acc: 0.77
epoch: 9, batch: 147, loss:0.898, acc: 0.74
epoch: 9, batch: 148, loss:0.882, acc: 0.75
epoch: 9, batch: 149, loss:0.953, acc: 0.66
epoch: 9, batch: 150, loss:0.839, acc: 0.68
epoch: 9, batch: 151, loss:0.783, acc: 0.76
epoch: 9, batch: 152, loss:0.946, acc: 0.72
epoch: 9, batch: 153, loss:0.853, acc: 0.71
epoch: 9, batch: 154, loss:0.947, acc: 0.69
epoch: 9, batch: 155, loss:1.063, acc: 0.61
epoch: 9, batch: 156, loss:0.911, acc: 0.70
epoch: 9, batch: 157, loss:0.783, acc: 0.70
epoch: 9, batch: 158, loss:0.791, acc: 0.76
epoch: 9, batch: 159, loss:1.144, acc: 0.61
epoch: 9, batch: 160, loss:0.880, acc: 0.68
epoch: 9, batch: 161, loss:0.970, acc: 0.68
epoch: 9, batch: 162, loss:0.891, acc: 0.70
epoch: 9, batch: 163, loss:1.055, acc: 0.66
epoch: 9, batch: 164, loss:1.089, acc: 0.66
epoch: 9, batch: 165, loss:0.777, acc: 0.72
epoch: 9, batch: 166, loss:0.878

epoch: 9, batch: 337, loss:0.843, acc: 0.71
epoch: 9, batch: 338, loss:0.962, acc: 0.63
epoch: 9, batch: 339, loss:0.861, acc: 0.74
epoch: 9, batch: 340, loss:0.899, acc: 0.71
epoch: 9, batch: 341, loss:0.831, acc: 0.72
epoch: 9, batch: 342, loss:0.930, acc: 0.68
epoch: 9, batch: 343, loss:1.084, acc: 0.67
epoch: 9, batch: 344, loss:0.807, acc: 0.73
epoch: 9, batch: 345, loss:0.652, acc: 0.80
epoch: 9, batch: 346, loss:1.062, acc: 0.66
epoch: 9, batch: 347, loss:0.937, acc: 0.69
epoch: 9, batch: 348, loss:0.989, acc: 0.70
epoch: 9, batch: 349, loss:0.972, acc: 0.69
epoch: 9, batch: 350, loss:0.847, acc: 0.64
epoch: 9, batch: 351, loss:0.924, acc: 0.69
epoch: 9, batch: 352, loss:0.962, acc: 0.66
epoch: 9, batch: 353, loss:0.731, acc: 0.70
epoch: 9, batch: 354, loss:0.903, acc: 0.69
epoch: 9, batch: 355, loss:0.779, acc: 0.76
epoch: 9, batch: 356, loss:1.077, acc: 0.62
epoch: 9, batch: 357, loss:0.946, acc: 0.64
epoch: 9, batch: 358, loss:0.923, acc: 0.69
epoch: 9, batch: 359, loss:0.958

epoch: 10, batch: 141, loss:0.912, acc: 0.70
epoch: 10, batch: 142, loss:0.663, acc: 0.76
epoch: 10, batch: 143, loss:0.906, acc: 0.71
epoch: 10, batch: 144, loss:0.881, acc: 0.72
epoch: 10, batch: 145, loss:0.725, acc: 0.77
epoch: 10, batch: 146, loss:0.948, acc: 0.66
epoch: 10, batch: 147, loss:0.853, acc: 0.71
epoch: 10, batch: 148, loss:0.885, acc: 0.72
epoch: 10, batch: 149, loss:0.771, acc: 0.75
epoch: 10, batch: 150, loss:0.943, acc: 0.72
epoch: 10, batch: 151, loss:0.776, acc: 0.74
epoch: 10, batch: 152, loss:0.920, acc: 0.71
epoch: 10, batch: 153, loss:0.834, acc: 0.69
epoch: 10, batch: 154, loss:1.000, acc: 0.62
epoch: 10, batch: 155, loss:0.813, acc: 0.74
epoch: 10, batch: 156, loss:0.793, acc: 0.73
epoch: 10, batch: 157, loss:0.927, acc: 0.66
epoch: 10, batch: 158, loss:0.793, acc: 0.69
epoch: 10, batch: 159, loss:1.028, acc: 0.73
epoch: 10, batch: 160, loss:1.050, acc: 0.62
epoch: 10, batch: 161, loss:0.834, acc: 0.69
epoch: 10, batch: 162, loss:1.035, acc: 0.67
epoch: 10,

epoch: 10, batch: 331, loss:0.809, acc: 0.78
epoch: 10, batch: 332, loss:0.952, acc: 0.66
epoch: 10, batch: 333, loss:0.919, acc: 0.66
epoch: 10, batch: 334, loss:0.797, acc: 0.80
epoch: 10, batch: 335, loss:0.951, acc: 0.66
epoch: 10, batch: 336, loss:0.810, acc: 0.74
epoch: 10, batch: 337, loss:0.834, acc: 0.62
epoch: 10, batch: 338, loss:0.853, acc: 0.73
epoch: 10, batch: 339, loss:0.891, acc: 0.72
epoch: 10, batch: 340, loss:0.894, acc: 0.70
epoch: 10, batch: 341, loss:0.857, acc: 0.72
epoch: 10, batch: 342, loss:0.730, acc: 0.74
epoch: 10, batch: 343, loss:0.894, acc: 0.66
epoch: 10, batch: 344, loss:0.890, acc: 0.73
epoch: 10, batch: 345, loss:0.840, acc: 0.75
epoch: 10, batch: 346, loss:0.790, acc: 0.73
epoch: 10, batch: 347, loss:0.910, acc: 0.68
epoch: 10, batch: 348, loss:1.011, acc: 0.67
epoch: 10, batch: 349, loss:0.939, acc: 0.70
epoch: 10, batch: 350, loss:0.779, acc: 0.72
epoch: 10, batch: 351, loss:0.856, acc: 0.69
epoch: 10, batch: 352, loss:0.991, acc: 0.66
epoch: 10,

## GradientTape

In [20]:
gt_model = functional_model(32, 32, 3)
gt_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                7376

In [21]:
loss_functions = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [22]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')
valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_acc')

In [23]:
@tf.function
def train_step(image, label):
    with tf.GradientTape() as tape:
        prediction = gt_model(image, training=True)
        loss = loss_functions(label, prediction)
    gradients = tape.gradient(loss, gt_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, gt_model.trainable_variables))
    
    train_loss(loss)
    train_acc(label, prediction)

In [24]:
@tf.function
def valid_step(image, label):
    prediction = gt_model(image, training=False)
    loss = loss_functions(label, prediction)
    
    valid_loss(loss)
    valid_acc(label, prediction)

In [25]:
EPOCHS = 10

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_acc.reset_states()
    valid_loss.reset_states()
    valid_acc.reset_states()
    
    for image, label in train_data:
        train_step(image, label)
    for image, label in valid_data:
        valid_step(image, label)
    
    print(f'epoch: {epoch + 1}, loss: {train_loss.result()}, acc: {train_acc.result()},\
          val_loss: {valid_loss.result()}, val_acc: {valid_acc.result()}')
          

epoch: 1, loss: 1.6966336965560913, acc: 0.3835799992084503,          val_loss: 1.4610493183135986, val_acc: 0.4846999943256378
epoch: 2, loss: 1.3626562356948853, acc: 0.515779972076416,          val_loss: 1.2925480604171753, val_acc: 0.5442000031471252
epoch: 3, loss: 1.221099615097046, acc: 0.5709999799728394,          val_loss: 1.1925580501556396, val_acc: 0.5838000178337097
epoch: 4, loss: 1.1305373907089233, acc: 0.6039800047874451,          val_loss: 1.1408371925354004, val_acc: 0.6007999777793884
epoch: 5, loss: 1.0624887943267822, acc: 0.6291000247001648,          val_loss: 1.0855562686920166, val_acc: 0.6208000183105469
epoch: 6, loss: 1.013546109199524, acc: 0.6487399935722351,          val_loss: 1.0775418281555176, val_acc: 0.6259999871253967
epoch: 7, loss: 0.9721930027008057, acc: 0.662559986114502,          val_loss: 1.037468671798706, val_acc: 0.6427000164985657
epoch: 8, loss: 0.9389998316764832, acc: 0.6748999953269958,          val_loss: 1.0163131952285767, val_acc: 